# Diabetes Prediction using Cost Sensitive Learning

**Highlights:**
> 1. Correlation analysis using Pearson coefficient
>
> 2. Classification of the imbalanced dataset using `class weighted` or `cost sensitive` learning
>
> 3. Rationale for choosing SVC with RBF kernel as an optimal classifier for the dataset
>
> 4. Results for each ML algorithm are presented after performing 5-fold cross validation based on F1-score

* [1. Import and Clean data](#import-and-clean-data)
    * [1.1. Checking for null values in the dataset](#checking-for-null-values-in-the-dataset)
* [2. Correlations in the data](#correlations-in-the-data)
* [3. Data Visualization](#data-visualization)
    * [3.1.  Frequency distribution](#frequency-distribution)
    * [3.2. Box Plots](#box-plots)
* [4. Data Preprocessing](#data-preprocessing)
    * [4.1. Train-Test split](#train-test-split)
    * [4.2. Standardization](#standardization)
* [5. Data Modeling](#data-modeling)
    * [5.1. Utility Functions](#utility-functions)
    * [5.2. Naive Bayes](#naive-bayes)
    * [5.3. Logistic Regression](#logistic-regression)
    * [5.4. K-Nearest Neighbors](#k-nearest-neighbors)
    * [5.5. Decision Tree](#decision-tree)
    * [5.6. Decision Trees with Bagging](#decision-trees-with-bagging)
    * [5.7. Random Forests](#random-forests)
    * [5.8. Decision Trees with AdaBoost](#decision-trees-with-adaboost)
    * [5.9. Linear SVC](#linear-svc)
    * [5.10. SVM with RBF kernel](#svm-with-rbf-kernel)
    * [5.11. XGBoost](#xgboost)
    * [5.12. CatBoost](#catboost)
* [6. Model Comparison](#model-comparison)
    * [6.1. Evaluation Metrics](#evaluation-metrics)
    * [6.2. ROC and PR Curves](#roc-and-pr-curves)

In [ ]:
import os, sys

import numpy as np
from scipy.stats import chi2_contingency
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

<a id="import-and-clean-data"></a>
# 1. Import and Clean data

In [ ]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df.head(2)

<a id="checking-for-null-values-in-the-dataset"></a>
## 1.1. Checking for null values in the dataset

In [ ]:
df.isnull().sum()

There are no null values in the dataset.

<a id="correlations-in-the-data"></a>
# 2. Correlations in the data

In [ ]:
df.describe().T

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm',
            cbar_kws={'aspect': 50}, square=True, ax=ax)
plt.xticks(rotation=30, ha='right');
plt.tight_layout()

***Inference:*** Glucose feature is higly correlated with the outcome (whether the patient has diabetes or not) which does make sense given diabetes is closely related to blood sugar levels.

<a id="data-visualization"></a>
# 3. Data Visualization

<a id="frequency-distribution"></a>
## 3.1.  Frequency distribution

In [ ]:
df_grouped = df.groupby(by='Outcome')

_, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

numeric_cols = list(df.columns[:-1])
ax_title_pairs = zip(axes.flat, numeric_cols)

for ax, title in ax_title_pairs:
    sns.distplot(df_grouped.get_group(0)[title],
                 bins=10, ax=ax, label='N')
    sns.distplot(df_grouped.get_group(1)[title],
                 bins=10, ax=ax, label='Y')
    ax.set_title(title)
    ax.set_xlabel('')
    ax.legend(title='Outcome')
    
plt.tight_layout()

<a id="box-plots"></a>
## 3.2. Box Plots

In [ ]:
_, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))
ax_title_pairs = zip(axes.flat, numeric_cols)

for ax, col in ax_title_pairs:
    sns.boxplot(x='Outcome', y=col, data=df, ax=ax)

plt.tight_layout()

<a id="data-preprocessing"></a>
# 4. Data Preprocessing
Data needs to be one-hot-encoded before applying machine learning models.

In [ ]:
x = df.iloc[:, :-1]
y = df['Outcome']

features = list(df.columns[:-1])

<a id="train-test-split"></a>
## 4.1. Train-Test split
CatBoost classifier does not require any kind of preprocessing. Therefore, we will use raw/ unmodified data (`x_train_cat, x_test_cat, y_train_cat, y_test_cat`) for CatBoost and preprocessed data (`x_train, x_test, y_train, y_test`) for all other classifiers.

In [ ]:
from sklearn.model_selection import train_test_split

data_splits = train_test_split(x, y, test_size=0.20, random_state=0,
                               shuffle=True, stratify=y)
x_train, x_test, y_train, y_test = data_splits


# For CatBoost and Naive Bayes
data_splits = train_test_split(x, y, test_size=0.20, random_state=0,
                               shuffle=True, stratify=y)
x_train_cat, x_test_cat, y_train_cat, y_test_cat = data_splits


list(map(lambda x: x.shape, [x, y, x_train, x_test, y_train, y_test]))

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
sns.countplot(x=y_test);

<a id="standardization"></a>
## 4.2. Standardization
We need to standardize the continuous or quantitative variables/ features before applying Machine Learning models. This is important because if we don't standardize the features, features with high variance that are orders of magnitude larger that others might dominate the model fitting process and causing the model unable to learn from other features (with lower variance) correctly as expected. 

***Also we need to standardize the data only after performing train-test split because if we standardize before splitting then there is a chance for some information leak from the test set into the train set. We always want the test set to be completely new to the ML models. [Read more](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)***

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder


## Column Transformer
transformers = [('standard_scaler', StandardScaler(), features)]
x_trans = ColumnTransformer(transformers, remainder='passthrough')

## Applying Column Transformer
x_train = x_trans.fit_transform(x_train)
x_test = x_trans.transform(x_test)

<a id="data-modeling"></a>
# 5. Data Modeling
Since the dataset is imbalanced we will be using class-weighted/ cost-sensitive learning. In cost-sensitive learning, a weighted cost function is used. Therefore, misclassifying a sample from the minority class will cost the classifiers more than misclassifying a sample from the majority class. In most of the Sklearn classifiers, cost-sensitive learning can be enabled by setting `class_weight='balanced'`.

<a id="utility-functions"></a>
## 5.1. Utility Functions

In [ ]:
import timeit
import pickle
import sys
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, accuracy_score
from sklearn.exceptions import NotFittedError

In [ ]:
def confusion_plot(matrix, labels=None):
    """ Display binary confusion matrix as a Seaborn heatmap """
    
    labels = labels if labels else ['Negative (0)', 'Positive (1)']
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    sns.heatmap(data=matrix, cmap='Blues', annot=True, fmt='d',
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel('PREDICTED')
    ax.set_ylabel('ACTUAL')
    ax.set_title('Confusion Matrix')
    plt.close()
    
    return fig

In [ ]:
def roc_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Receiver Operating Characteristic (ROC) curve 
        Set `compare=True` to use this function to compare classifiers. """
    
    fpr, tpr, thresh = roc_curve(y_true, y_probs, drop_intermediate=False)
    auc = round(roc_auc_score(y_true, y_probs), 2)
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    label = ' '.join([label, f'({auc})']) if compare else None
    sns.lineplot(x=fpr, y=tpr, ax=axis,
                 estimator=None, label=label)
    
    if compare:
        axis.legend(title='Classifier (AUC)', loc='lower right')
    else:
        axis.text(0.72, 0.05, f'AUC = { auc }', fontsize=12,
                  bbox=dict(facecolor='green', alpha=0.4, pad=5))
            
        # Plot No-Info classifier
        axis.fill_between(fpr, fpr, tpr, alpha=0.3, edgecolor='g',
                          linestyle='--', linewidth=2)
        
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('ROC Curve')
    axis.set_xlabel('False Positive Rate [FPR]\n(1 - Specificity)')
    axis.set_ylabel('True Positive Rate [TPR]\n(Sensitivity or Recall)')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def precision_recall_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Precision-Recall curve.
        Set `compare=True` to use this function to compare classifiers. """
    
    p, r, thresh = precision_recall_curve(y_true, y_probs)
    p, r, thresh = list(p), list(r), list(thresh)
    p.pop()
    r.pop()
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    
    if compare:
        sns.lineplot(r, p, estimator=None,
                     ax=axis, label=label)
        axis.set_xlabel('Recall')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')
    else:
        sns.lineplot(thresh, p, estimator=None,
                     label='Precision', ax=axis)
        axis.set_xlabel('Threshold')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')

        axis_twin = axis.twinx()
        sns.lineplot(thresh, r, estimator=None,
                     color='limegreen', label='Recall', ax=axis_twin)
        axis_twin.set_ylabel('Recall')
        axis_twin.set_ylim(0, 1)
        axis_twin.legend(bbox_to_anchor=(0.24, 0.18))
    
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('Precision Vs Recall')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def feature_importance_plot(importances, feature_labels, ax=None):
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1, figsize=(5, 10))
    sns.barplot(x=importances, y=feature_labels, ax=axis)
    axis.set_title('Feature Importance Measures')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def train_clf(clf, x_train, y_train, sample_weight=None, refit=False):
    train_time = 0
    
    try:
        if refit:
            raise NotFittedError
        y_pred_train = clf.predict(x_train)
    except NotFittedError:
        start = timeit.default_timer()
        
        if sample_weight is not None:
            clf.fit(x_train, y_train, sample_weight=sample_weight)
        else:
            clf.fit(x_train, y_train)
        
        end = timeit.default_timer()
        train_time = end - start
        
        y_pred_train = clf.predict(x_train)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    return clf, y_pred_train, train_acc, train_time

In [ ]:
def model_memory_size(clf):
    return sys.getsizeof(pickle.dumps(clf))

In [ ]:
def report(clf, x_train, y_train, x_test, y_test, display_scores=[],
           sample_weight=None, refit=False, importance_plot=False,
           confusion_labels=None, feature_labels=None, verbose=True):
    """ Trains the passed classifier if not already trained and reports
        various metrics of the trained classifier """
    
    dump = dict()
    
    ## Train if not already trained
    clf, train_predictions, \
    train_acc, train_time = train_clf(clf, x_train, y_train,
                                      sample_weight=sample_weight,
                                      refit=refit)
    ## Testing
    start = timeit.default_timer()
    test_predictions = clf.predict(x_test)
    end = timeit.default_timer()
    test_time = end - start
    
    test_acc = accuracy_score(y_test, test_predictions)
    y_probs = clf.predict_proba(x_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_probs)
        
    ## Additional scores
    scores_dict = dict()
    for func in display_scores:
        scores_dict[func.__name__] = [func(y_train, train_predictions),
                                      func(y_test, test_predictions)]
        
    ## Model Memory
    model_mem = round(model_memory_size(clf) / 1024, 2)
    
    print(clf)
    print("\n=============================> TRAIN-TEST DETAILS <======================================")
    
    ## Metrics
    print(f"Train Size: {x_train.shape[0]} samples")
    print(f" Test Size: {x_test.shape[0]} samples")
    print("---------------------------------------------")
    print(f"Training Time: {round(train_time, 3)} seconds")
    print(f" Testing Time: {round(test_time, 3)} seconds")
    print("---------------------------------------------")
    print("Train Accuracy: ", train_acc)
    print(" Test Accuracy: ", test_acc)
    print("---------------------------------------------")
    
    if display_scores:
        for k, v in scores_dict.items():
            score_name = ' '.join(map(lambda x: x.title(), k.split('_')))
            print(f'Train {score_name}: ', v[0])
            print(f' Test {score_name}: ', v[1])
            print()
        print("---------------------------------------------")
    
    print(" Area Under ROC (test): ", roc_auc)
    print("---------------------------------------------")
    print(f"Model Memory Size: {model_mem} kB")
    print("\n=============================> CLASSIFICATION REPORT <===================================")
    
    ## Classification Report
    clf_rep = classification_report(y_test, test_predictions, output_dict=True)
    
    print(classification_report(y_test, test_predictions,
                                target_names=confusion_labels))
    
    
    if verbose:
        print("\n================================> CONFUSION MATRIX <=====================================")
    
        ## Confusion Matrix HeatMap
        display(confusion_plot(confusion_matrix(y_test, test_predictions),
                               labels=confusion_labels))
        print("\n=======================================> PLOTS <=========================================")


        ## Variable importance plot
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
        roc_axes = axes[0, 0]
        pr_axes = axes[0, 1]
        importances = None

        if importance_plot:
            if not feature_labels:
                raise RuntimeError("'feature_labels' argument not passed "
                                   "when 'importance_plot' is True")

            try:
                importances = pd.Series(clf.feature_importances_,
                                        index=feature_labels) \
                                .sort_values(ascending=False)
            except AttributeError:
                try:
                    importances = pd.Series(clf.coef_.ravel(),
                                            index=feature_labels) \
                                    .sort_values(ascending=False)
                except AttributeError:
                    pass

            if importances is not None:
                # Modifying grid
                grid_spec = axes[0, 0].get_gridspec()
                for ax in axes[:, 0]:
                    ax.remove()   # remove first column axes
                large_axs = fig.add_subplot(grid_spec[0:, 0])

                # Plot importance curve
                feature_importance_plot(importances=importances.values,
                                        feature_labels=importances.index,
                                        ax=large_axs)
                large_axs.axvline(x=0)

                # Axis for ROC and PR curve
                roc_axes = axes[0, 1]
                pr_axes = axes[1, 1]
            else:
                # remove second row axes
                for ax in axes[1, :]:
                    ax.remove()
        else:
            # remove second row axes
            for ax in axes[1, :]:
                ax.remove()


        ## ROC and Precision-Recall curves
        clf_name = clf.__class__.__name__
        roc_plot(y_test, y_probs, clf_name, ax=roc_axes)
        precision_recall_plot(y_test, y_probs, clf_name, ax=pr_axes)

        fig.subplots_adjust(wspace=5)
        fig.tight_layout()
        display(fig)
    
    ## Dump to report_dict
    dump = dict(clf=clf, accuracy=[train_acc, test_acc], **scores_dict,
                train_time=train_time, train_predictions=train_predictions,
                test_time=test_time, test_predictions=test_predictions,
                test_probs=y_probs, report=clf_rep, roc_auc=roc_auc,
                model_memory=model_mem)
    
    return clf, dump

In [ ]:
def compare_models(y_test=None, clf_reports=[], labels=[], score='accuracy'):
    """ Compare evaluation metrics for the True Positive class [1] of 
        binary classifiers passed in the argument and plot ROC and PR curves.
        
        Arguments:
        ---------
        y_test: to plot ROC and Precision-Recall curves
         score: is the name corresponding to the sklearn metrics
        
        Returns:
        -------
        compare_table: pandas DataFrame containing evaluated metrics
                  fig: `matplotlib` figure object with ROC and PR curves """

    
    ## Classifier Labels
    default_names = [rep['clf'].__class__.__name__ for rep in clf_reports]
    clf_names =  labels if len(labels) == len(clf_reports) else default_names
    
    
    ## Compare Table
    table = dict()
    index = ['Train ' + score, 'Test ' + score, 'Overfitting', 'ROC Area',
             'Precision', 'Recall', 'F1-score', 'Support']
    for i in range(len(clf_reports)):
        scores = [round(i, 3) for i in clf_reports[i][score]]
        
        roc_auc = clf_reports[i]['roc_auc']
        
        # Get metrics of True Positive class from sklearn classification_report
        true_positive_metrics = list(clf_reports[i]['report']["1"].values())
        
        table[clf_names[i]] = scores + [scores[1] < scores[0], roc_auc] + \
                              true_positive_metrics
    
    table = pd.DataFrame(data=table, index=index)
    
    
    ## Compare Plots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    
    # ROC and Precision-Recall
    for i in range(len(clf_reports)):
        clf_probs = clf_reports[i]['test_probs']
        roc_plot(y_test, clf_probs, label=clf_names[i],
                 compare=True, ax=axes[0])
        precision_recall_plot(y_test, clf_probs, label=clf_names[i],
                              compare=True, ax=axes[1])
    # Plot No-Info classifier
    axes[0].plot([0,1], [0,1], linestyle='--', color='green')
        
    fig.tight_layout()
    plt.close()
    
    return table.T, fig

<a id="naive-bayes"></a>
## 5.2. Naive Bayes
The fundamental assumption made by Naive Bayes regarding the data is ***class conditional independence of features***. Sklearn provides different variants of Naive Bayes depending on whether the features follow a categorical distribution (CategoricalNB), normal distribution (GaussianNB), bernoulli distribution (BernoulliNB), multinomial distribution (MultinomialNB). 
 
Since majority of the features are continuous and follow an approximately normal distribution, we will use GaussianNB.

In [ ]:
from sklearn.metrics import f1_score
score_func = f1_score

In [ ]:
from sklearn.naive_bayes import GaussianNB 

confusion_lbs = ['No Diabetes', 'Have Diabetes']  # (0, 1)

nb_clf = GaussianNB()

nb_clf, nb_report = report(nb_clf, x_train, y_train,
                           x_test, y_test, refit=True,
                           display_scores=[score_func],
                           confusion_labels=confusion_lbs)

<a id="logistic-regression"></a>
## 5.3. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

logit_cv = LogisticRegressionCV(class_weight='balanced', cv=5, max_iter=500,
                                scoring='recall', penalty='l2', solver='liblinear',
                                n_jobs=-1, random_state=0, refit=True, verbose=0)

logit_cv, logit_report = report(logit_cv, x_train, y_train,
                                x_test, y_test,
                                display_scores=[score_func],
                                refit=True, importance_plot=True,
                                feature_labels=features,
                                confusion_labels=confusion_lbs)

<a id="k-nearest-neighbors"></a>
## 5.4. K-Nearest Neighbors
KNN estimator in Scikit-learn does not provide a way to pass class-weights to enable cost-sensitive/ class-weighted learning.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=15, p=2,
                           weights='distance', n_jobs=-1)

knn, knn_report = report(knn, x_train, y_train,
                         x_test, y_test,
                         display_scores=[score_func],
                         importance_plot=True,
                         feature_labels=features,
                         confusion_labels=confusion_lbs)

<a id="decision-tree"></a>
## 5.5. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(class_weight='balanced',
                                       criterion='entropy',
                                       max_depth=5,
                                       random_state=0)

decision_tree, decision_tree_report = report(decision_tree, x_train, y_train,
                                             x_test, y_test,
                                             display_scores=[score_func],
                                             importance_plot=True,
                                             feature_labels=features,
                                             confusion_labels=confusion_lbs)

<a id="decision-trees-with-bagging"></a>
## 5.6. Decision Trees with Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_dtree = DecisionTreeClassifier(max_depth=2, class_weight='balanced',
                                       criterion='entropy', random_state=0)

bagging_clf = BaggingClassifier(base_estimator=bagging_dtree,
                                max_samples=0.1873, n_estimators=140,
                                n_jobs=-1, random_state=0)

bagging_clf, bagging_clf_report = report(bagging_clf, x_train, y_train,
                                         x_test, y_test,
                                         display_scores=[score_func],
                                         feature_labels=features,
                                         confusion_labels=confusion_lbs)

<a id="random-forests"></a>
## 5.7. Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                                       max_depth=3, max_samples=0.54, n_estimators=160,
                                       n_jobs=-1, random_state=0)

random_forest, random_forest_report = report(random_forest, x_train, y_train,
                                             x_test, y_test,
                                             display_scores=[score_func],
                                             importance_plot=True,
                                             feature_labels=features,
                                             confusion_labels=confusion_lbs)

<a id="decision-trees-with-adaboost"></a>
## 5.8. Decision Trees with AdaBoost
The default base estimator for `AdaBoostClassifier` is `DecisionTreeClassifier(max_depth=1)`

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

boosting_dtree = DecisionTreeClassifier(class_weight='balanced',
                                        criterion='entropy',
                                        max_depth=1, random_state=0)

adaboot = AdaBoostClassifier(base_estimator=boosting_dtree,
                             n_estimators=40, learning_rate=0.073,
                             random_state=0)

adaboot, adaboot_report = report(adaboot, x_train, y_train,
                                 x_test, y_test,
                                 display_scores=[score_func],
                                 importance_plot=True,
                                 feature_labels=features,
                                 confusion_labels=confusion_lbs)

<a id="linear-svc"></a>
## 5.9. Linear SVC

In [ ]:
from sklearn.svm import SVC

linear_svc = SVC(C=0.026, kernel='linear', probability=True,
                 class_weight='balanced', random_state=0)

linear_svc, linear_svc_report = report(linear_svc, x_train, y_train,
                                       x_test, y_test,
                                       display_scores=[score_func],
                                       importance_plot=True,
                                       feature_labels=features,
                                       confusion_labels=confusion_lbs)

<a id="svm-with-rbf-kernel"></a>
## 5.10. SVM with RBF kernel

In [ ]:
rbf_svc = SVC(C=0.585, kernel='rbf', probability=True,
              class_weight='balanced', random_state=0)

rbf_svc, rbf_svc_report = report(rbf_svc, x_train, y_train,
                                 x_test, y_test,
                                 display_scores=[score_func],
                                 importance_plot=True,
                                 feature_labels=features,
                                 confusion_labels=confusion_lbs)

<a id="xgboost"></a>
## 5.11. XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

## Compute `class_weights` using sklearn
cls_weight = (y_train.shape[0] - np.sum(y_train)) / np.sum(y_train)

params = {'learning_rate': 0.09995835505176981, 'num_boost_round': 622,
          'max_depth': 8, 'min_child_weight': 21.477702263891228,
          'subsample': 0.7783353885462759, 'colsample_bytree': 0.8808385233283204}

xgb_clf = XGBClassifier(**params, scale_pos_weight=cls_weight,
                        random_state=0, n_jobs=-1)
xgb_clf.fit(x_train, y_train, eval_set=[(x_train, y_train)],
            eval_metric='aucpr', early_stopping_rounds=10,
            verbose=False);

xgb_clf, xgb_report = report(xgb_clf, x_train, y_train,
                             x_test, y_test,
                             display_scores=[score_func],
                             importance_plot=True,
                             feature_labels=features,
                             confusion_labels=confusion_lbs)

<a id="catboost"></a>
## 5.12. CatBoost
Cat boost performs better without One-hot encoding because it performs an internal categorical encoding that is similar to Leave One Out Encoding (LOOE). So, we can give the dataframe as input to the catboost classifier.

In [ ]:
from catboost import CatBoostClassifier

# Basic working

catboost_clf = CatBoostClassifier(cat_features=[],
                                  l2_leaf_reg=120, depth=6,
                                  auto_class_weights='Balanced',
                                  iterations=200, learning_rate=0.16,
                                  use_best_model=True,
                                  early_stopping_rounds=150,
                                  eval_metric='Recall', random_state=0)

catboost_clf.fit(x_train_cat, y_train, 
                 eval_set=(x_train_cat, y_train),
                 verbose=False)


catboost_clf, catboost_report = report(catboost_clf, x_train_cat, y_train,
                                       x_test_cat, y_test,
                                       display_scores=[score_func],
                                       importance_plot=True,
                                       feature_labels=features,
                                       confusion_labels=confusion_lbs)

<a id="model-comparison"></a>
# 6. Model Comparison

In [ ]:
report_list = [nb_report, logit_report, knn_report, decision_tree_report,               
               bagging_clf_report, random_forest_report, adaboot_report,
               xgb_report, linear_svc_report, rbf_svc_report, catboost_report]
clf_labels = [rep['clf'].__class__.__name__ for rep in report_list]
clf_labels[-3], clf_labels[-2] = 'Linear SVC', 'RBF SVC'

<a id="evaluation-metrics"></a>
## 6.1. Evaluation Metrics
Since the dataset is imbalanced, 'F1-score' is a more appropriate evaluation metric than accuracy.

In [ ]:
compare_table, compare_plot = compare_models(y_test, clf_reports=report_list,
                                             labels=clf_labels, score=score_func.__name__)

In [ ]:
compare_table.sort_values(by=['Overfitting', 'F1-score'],
                          ascending=[True, False])

***Inference:*** We can see that among the classifiers that do not overfit (based on F1 score), SVC with RBF kernel has the highest F1 score.

<a id="roc-and-pr-curves"></a>
## 6.2. ROC and PR Curves

In [ ]:
compare_plot

**Thank You!!**